In [10]:
from llm.llm.chatgpt import ChatGPT
from llm.llm.gemini import Gemini
from setup_db import create_chroma_db, connect_to_db, execute_query, DBHUB
from branch_reasoning import *

In [11]:
llm = Gemini()

In [12]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [13]:
conn = connect_to_db(db_name, user, password, host, port)

Connecting to database test_db, postgres...


In [14]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'company_name_chroma'
persist_directory = 'data/company_name_chroma'
vector_db_company = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'sql_query'
persist_directory = 'data/sql_query'
vector_db_sql = create_chroma_db(collection_chromadb, persist_directory)

In [15]:
vector_db_company.similarity_search('company', k=5)

[Document(page_content='SSI Securities Corporation', metadata={'lang': 'en', 'stock_code': 'SSI'}),
 Document(page_content='SSI', metadata={'lang': 'vi', 'stock_code': 'SSI'}),
 Document(page_content='Masan Group Corporation', metadata={'lang': 'en', 'stock_code': 'MSN'}),
 Document(page_content='VIC', metadata={'lang': 'vi', 'stock_code': 'VIC'}),
 Document(page_content='SSI Securities', metadata={'lang': 'en', 'stock_code': 'SSI'})]

In [16]:
db = DBHUB(conn, bank_vector_store, none_bank_vector_store, vector_db_company, vector_db_sql)

In [17]:
prompt =  "Calculate ROA, ROE of all the company which are owned by VinGroup"

In [18]:
history = llm_branch_reasoning(llm, prompt, db, verbose=True)

prompt_token_count: 395
candidates_token_count: 160
total_token_count: 555

Branch reasoning response: 
```json
{
  "steps": [
    "Step 1: Identify all companies owned by VinGroup by joining the `company_info` and `sub_and_shareholder` tables.  This will yield a list of company IDs or stock codes wholly or partially owned by VinGroup.",
    "Step 2:  Calculate ROA and ROE for each company identified in Step 1. This requires joining the results from Step 1 with either the `bank_financial_report` or `non_bank_financial_report` tables based on the company type (obtained from `company_info`), and performing the necessary calculations using the relevant financial data (e.g., net income, total assets, shareholder equity)."
  ]
}
```

Step 0: Extract company name
prompt_token_count: 71
candidates_token_count: 20
total_token_count: 91

Get stock code based on company name response: 
```json
{
  "company_name": ["VinGroup"]
}
```

type of company_info_df: <class 'pandas.core.frame.DataFrame'>


TypeError: cannot concatenate object of type '<class 'str'>'; only Series and DataFrame objs are valid